# Lựa chọn đặc trưng and LASSO

Trong notebook này, chúng ta sẽ sử dụng LASSO để lựa chọn đặc trưng, xây dựng solver đã triển khai trước cho LASSO (trong sklearn). Cụ thể, chúng ta sẽ:  
* Chạy LASSO với các L1 penalty khác nhau.
* Chọn L1 penalty tốt nhất sử dụng tập kiểm định.
* Chọn L1 penalty tốt nhất sử dụng tập kiểm định với ràng buộc bổ sung về kích thước tập con.

Trong bài tập tiếp theo, chúng ta sẽ tạo LASSO solver sử dụng coordinate descent.

## Thư viện

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sklearn
import pandas
import numpy as np

## Load tập dữ liệu doanh số bán nhà

Tập dữ liệu từ doanh số bán nhà quận King, Seatle, WA.

In [3]:
full_data = pandas.read_csv("/content/drive/MyDrive/Colab_Notebooks/Bai tap lap/Mon_hoc_2/kc_house_data.csv", index_col=0)

## Tạo các đặc trung mới

Như ở lab 2 (*Lab-2.ipynb*), chúng ta sẽ xem xét các đặc trưng có các biến đổi đầu vào.

In [4]:
from math import log, sqrt
full_data['sqft_living_sqrt'] = full_data['sqft_living'].map(sqrt)
full_data['sqft_lot_sqrt'] = full_data['sqft_lot'].map(sqrt)
full_data['bedrooms_square'] = full_data['bedrooms'] ** 2

# Trong tập dữ liệu, 'floors' được xác định là type string,
# nên chúng ta sẽ chuyển chúng thành float trước khi tạo đặc trưng mới.
full_data['floors'] = full_data['floors'].astype(float)
full_data['floors_square'] = full_data['floors'] ** 2

* Bình phương bedrooms sẽ tăng phân tách giữa ít phòng ngủ (chẳng hạn 1) và nhiều phòng ngủ (chẳng hạn 4) vì 1^2 = 1 còn 4^2 = 16. Do đó, biến này sẽ ảnh hưởng lớn tới các ngôi nhà có nhiều phòng ngủ.
* Mặt khác, căn bậc hai của sqft_living sẽ giảm phân tách giữa nhà lớn và nhà nhỏ. Chủ ngôi nhà cũng sẽ không vui hơn nếu nhà rộng gấp đôi.

# Tìm hiểu trọng số hồi quy với L1 penalty

Hãy khớp mô hình với tất cả đặc trưng hiện có cộng với các đặc trưng vừa tạo.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Áp dụng L1 penalty cần thêm tham số (`alpha=l1_penalty`) cho mô hình  `Lasso` của Sklearn. (Các công cụ khác cũng phân tách các triển khai của LASSO). Khá giống Hồi quy Ridge/L2, các đặc trưng cũng cần được co giãn để đảm bảo đồng đều ở giữa.

In [6]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
l1_penalty=5e4
full_features = scaler.fit_transform(full_data[all_features].values)
full_labels = full_data['price'].values
model = Lasso(alpha=l1_penalty).fit(full_features, full_labels)

Tìm các đặc trưng có trọng số khác 0.

In [9]:
# Bạn có biết numpy cũng có boolean selector tích hợp sẵn?
print(model.coef_)

[     0.              0.              0.         132571.09360631
      0.             -0.             -0.              0.
      0.          14623.33961421  29004.06421249      0.
  90207.54789031      0.              0.         -10722.34912003
      0.        ]


In [17]:
mask = model.coef_ !=0
print(mask)
for index, i in enumerate(mask):
      if i == 1: print(all_features[index])

[False False False  True False False False False False  True  True False
  True False False  True False]
sqft_living
waterfront
view
grade
yr_built


Lưu ý rằng phần lớn trọng số được đặt thành 0. Vì vậy, bằng cách đặt L1 penalty đủ lớn, chúng ta có thể thực hiện lựa chọn tập con.

***QUIZ***:
Theo list các trọng số này, những đặc trưng nào đã được chọn?

# Lựa chọn L1 penalty

Để tìm một L1 penalty tốt, chúng ta sẽ khám phá nhiều giá trị sử dụng tập kiểm định. Hãy chia dữ liệu thành tập huấn luyện, tập kiểm định và tập kiểm tra:
* Chia dữ liệu bán hàng thành 2 tập: tập huấn luyện và tập kiểm tra (9/1)
* Chia tiếp tập huấn luyện thành 2 tập: tập huấn luyện và kiểm định (5/5)

Hãy dùng seed = 1 để có cùng kết quả!

In [18]:
from sklearn.model_selection import train_test_split
# Cookie cho những ai cần copy cell

In [19]:
train_val_data, test_data = train_test_split(full_data, train_size=0.9, test_size=0.1, random_state=1)
train_data, val_data = train_test_split(train_val_data,train_size=0.5, test_size=0.5, random_state=1)

Tiếp theo, chúng ta sẽ viết một vòng lặp như sau:
* Với `l1_penalty` trong phạm vi 21 bước giữa [1, 10^9] (sử dụng `np.logspace(0, 9, num=21)`.)
    * Khớp mô hình hồi quy với `l1_penalty` trong dữ liệu HUẤN LUYỆN. Chỉ định `alpha=l1_penalty` trong tham số.
    * Tính RSS trên dữ liệu KIỂM ĐỊNH (sử dụng `.predict()`) cho `l1_penalty`
* Báo lại `l1_penalty` nào cho RSS thấp nhất trong dữ liệu KIỂM ĐỊNH.

In [30]:
# Mức độ trợ giúp: Bình thường.
l1_penalty = np.logspace(0,9,num=21)

data_train_feature = scaler.fit_transform(train_data[all_features].values)
data_train_label = train_data['price'].values

data_val_feature = scaler.fit_transform(val_data[all_features].values)
data_val_label = val_data['price'].values

data_test_feature = scaler.fit_transform(test_data[all_features].values)
data_test_label = test_data['price'].values

model_list = []
RSS_val=[]

for i in l1_penalty:
    model = Lasso(alpha=i).fit(data_train_feature, data_train_label)
    model_list.append(model)
    value_predict = model.predict(data_val_feature)
    error = value_predict - data_val_label
    RSS_val.append(error@error)
RSS_val =np.array(RSS_val)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.461e+13, tolerance: 1.275e+11
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.462e+13, tolerance: 1.275e+11
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.465e+13, tolerance: 1.275e

501.18723362727246


***QUIZ:*** Giá trị tốt nhất cho `l1_penalty` là bao nhiêu?

In [31]:
# Qua quan sát hay tính toán?
index_min = RSS_val.argmin()
print(index_min)
print(l1_penalty[index_min])

6
501.18723362727246


***QUIZ***
Với giá trị L1 penalty này, chúng ta có bao nhiêu trọng số khác 0?

In [43]:
# Thú vị phải không?
print(model_list[6].coef_)
print(model_list[6].intercept_)

[ -14621.35244604    5822.44897681   41651.73155433  435052.0814051
 -411212.35081184   20338.49433071  -39049.28396132  -16422.18418574
   28671.19298684   41933.17550451   29201.19518856   17746.80516728
  151076.03197317   96365.70236264   54920.38773254  -93814.60567627
    5709.03763779]
538291.4423650386


# Limit the number of nonzero weights Giới hạn số trọng số khác 0

Sẽ ra sao nếu chúng ta muốn giới hạn, 5 đặc trưng chẳng hạn? Điều này quan trọng nếu chúng ta muốn suy ra "quy tắc ngón tay cái" --- mô hình có thể diễn giải chỉ với một vài đặc trưng.

Trong phần này, chúng ta sẽ triển khai quy trình đơn giản gồm 2 giai đoạn :
1. Thăm dò phạm vi lớn các giá trị `l1_penalty` để tìm vùng các giá trị `l1_penalty` hẹp hơn mà mô hình chắc chắn sẽ có số lượng trọng số khác 0 mong muốn.
2. Thăm dò tiếp vùng hẹp đã thấy để tìm gái trị tốt cho `l1_penalty` đạt được độ thưa thớt mong muốn. Ở đây chúng ta sử dụng tập kiểm định để chọn giá trị tốt nhất cho `l1_penalty`.

In [41]:
max_nonzeros = 5

## Khám phá phạm vi giá trị lớn hơn để tìm phạm vi hẹp với độ thưa thớt mong muốn

Hãy xác định một loạt các `l1_penalty_values` có thể:

In [52]:
l1_penalty_values = np.logspace(3, 5, num=21)

Giờ hãy triển khi vòng lặp tìm kiếm trong không gian có các giá trị `l1_penalty` có thể:

* Với `l1_penalty` trong `np.logspace(3, 5, num=21)`:
    * Khớp mô hình hồi quy với `l1_penalty` đã biết trong dữ liệu HUẤN LUYỆN. Chỉ định `alpha=l1_penalty` trong tham số.
    * Trích xuất trọng số của mô hình và đếm số trọng số khác 0. Lưu con số này vào một list.
        * Gợi ý: `model.coef_` cho các tham số/hệ số đã tìm thấy (intercept) ở dạng mảng numpy. Sau đó có thể dùng array\[condition\] cho list các giá trị truyền điều kiện, hoặc chỉ dùng `np.count_nonzero()` có sẵn.

In [56]:
# Hoặc có thể thực hiện với python thuần túy. Nó không ảnh hưởng đáng kể tới chất lượng.
model_list_2 = []
model_coef = {}
model_count_nonzero = []
for i in l1_penalty_values:
    model = Lasso(alpha=i).fit(data_train_feature, data_train_label)
    model_list_2.append(model)
    name = 'model_' + str(i)
    model_coef[name] = list(model.coef_)
    model_count_nonzero.append(np.count_nonzero(model.coef_))
print(model_coef)
print(model_count_nonzero)


{'model_1000.0': [-15120.241951055126, 5134.111634345809, 39848.789090764534, 502423.8530733707, -370763.7411615777, 15089.031555196498, -33286.19301867534, -0.0, 13250.095816808585, 41595.68564431896, 29692.24935799432, 16771.804539995133, 150119.27470013325, 0.0, 3706.812576124427, -93890.8470231672, 5048.52230438773], 'model_1258.9254117941675': [-15295.232356588574, 4741.784396686107, 39292.778840105515, 480849.7549177148, -348243.6607738056, 12544.09143612364, -30535.78156638679, -0.0, 13183.529084169086, 41445.659317866666, 29912.761456833872, 16173.86749375976, 149509.05259426153, 0.0, 3425.4735575889813, -94102.37821101972, 4715.09629423114], 'model_1584.893192461114': [-15516.804187350586, 4248.53832304876, 38594.26661613436, 453676.9506226394, -319881.9610217005, 9334.239277519764, -27067.205395860736, -0.0, 13100.538428803726, 41256.682125864405, 30190.413442744946, 15420.724563811957, 148741.3942764866, 0.0, 3071.8594530294004, -94369.64144087445, 4295.063966513791], 'model

Trong phạm vi lớn này, chúng ta có thể tìm 2 đầu phạm vi hẹp mong muốn của `l1_penalty`. Ở một đầu, các giá trị `l1_penalty` có quá ít giá trị khác 0, còn đầu kia lại có quá nhiều giá trị khác 0.

Hãy tìm:
* `l1_penalty` nhỏ nhất có các số khác 0 bằng `max_nonzeros` (nếu chọn penalty nhỏ hơn giá trị này chắc chắn sẽ có rất nhiều trọng số khác 0).
    * Lưu giá trị này trong biến `l1_penalty_min` (sẽ sử dụng nó sau).
* `l1_penalty` lớn nhất có các số khác 0 bằng `max_nonzeros` (nếu chọn penalty lớn hơn giá trị này chắc chắn sẽ có rất ít trọng số khác 0).
    * Lưu giá trị này trong biến `l1_penalty_max` (sẽ sử dụng nó sau).


*Gợi ý: có nhiều cách để thực hiện, chẳng hạn:*
* Lập trình trong vòng lặp trên.
* Tạo một list với số lượng khác 0 cho từng giá trị `l1_penalty` và kiểm tra nó để tìm ranh giới thích hợp.

In [57]:
tem_list = []
for index, i in enumerate(model_count_nonzero):
    if i == max_nonzeros: tem_list.append(l1_penalty_values[index])
tem_list = np.array(tem_list)
l1_penalty_min = tem_list.min()
l1_penalty_max = tem_list.max()

In [58]:
print(tem_list)
print(l1_penalty_min,'/n',l1_penalty_max)

[25118.8643151  31622.77660168 39810.71705535 50118.72336273]
25118.864315095823 /n 50118.72336272725


***QUIZ.*** Chúng ta tìm thấy các giá trị nào lần lượt cho `l1_penalty_min` và `l1_penalty_max`?

## Khám phá phạm vi nhỏ các giá trị để tìm giải pháp với đúng số lượng các số khác 0 có RSS trong tập kiểm định nhỏ nhất

Chúng ta sẽ khám phá vùng hẹp các giá trị `l1_penalty` đã tìm thấy:

In [59]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* Với `l1_penalty` trong `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Khớp mô hình hồi quy với `l1_penalty` đã biết trong dữ liệu HUẤN LUYỆN. Chỉ định `alpha=l1_penalty`.
    * Đo lường RSS của mô hình đã tìm hiểu trong tập KIỂM ĐỊNH.

Tìm mô hình có RSS nhỏ nhất trong tập KIỂM ĐỊNH và độ thưa thớt *bằng*  `max_nonzeros`.

In [60]:
from numpy.core.fromnumeric import argmin
# Xem quiz bên dưới
model_list_3 = []
RSS_val_3 = []


for i in np.linspace(l1_penalty_min,l1_penalty_max,20):
      model = Lasso(alpha=i).fit(data_train_feature, data_train_label)
      model_list_3.append(model)
      value_predict = model.predict(data_val_feature)
      error = value_predict - data_val_label
      RSS_val_3.append(error@error)

RSS_val_3 = np.array(RSS_val_3)


In [71]:
print(RSS_val_3)
mm = RSS_val_3.argsort()
non_zero_list = []

for i in range(len(RSS_val_3)):
    non_zero_list.append(np.count_nonzero(model_list_3[i].coef_))
print(non_zero_list)

vitri = 0

for i in mm:
    if non_zero_list[i] == max_nonzeros:
        vitri = i
        break
    else: pass
print(RSS_val_3[vitri])

[4.91275148e+14 4.94382860e+14 4.97634381e+14 5.01029454e+14
 5.04568079e+14 5.08250255e+14 5.12075984e+14 5.16045265e+14
 5.20158097e+14 5.24414483e+14 5.28814420e+14 5.33357909e+14
 5.38044952e+14 5.42875550e+14 5.47849702e+14 5.52967405e+14
 5.58228662e+14 5.63633471e+14 5.69181832e+14 5.74873747e+14]
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
491275148416788.06


In [63]:
print("o hinh tot nhat: {} co gai tri l1_penalty: {}".format(index_min_3,l1_penalty_min ))

o hinh tot nhat: 0 co gai tri l1_penalty: 25118.864315095823


***QUIZ***
1. Giá trị của `l1_penalty` trong phạm vi hẹp hơn có RSS thấp nhất trong tập KIỂM ĐỊNH và độ thưa *bằng* `max_nonzeros` là?
2. Các đặc trung nào trong mô hình này có các hệ số khác 0?

In [72]:
print(" giá trị của l1_penalty cần chính là L1_penalty_min : 25118.864315095823")

mask_3 = model_list_3[index_min_3].coef_ != 0
print(model_list_3[index_min_3].coef_)

for index, i in enumerate(mask_3):
    if i == 1: print(all_features[index])

 giá trị của l1_penalty cần chính là L1_penalty_min : 25118.864315095823
[    -0.              0.              0.         143559.23704386
      0.             -0.             -0.              0.
      0.          26144.47317615  31316.2925758       0.
 121438.23477256      0.              0.         -53125.49162964
      0.        ]
sqft_living
waterfront
view
grade
yr_built
